In [1]:
import re
import pandas as pd
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET

In [2]:
description_text = re.compile("\<description id\=\"description\"\>(.*?)\<\/description\>")
grant_id = re.compile('file\=\"([U][S]\w\w\d{6})\-\d{8}\.XML\"')

In [3]:
# some extra care with description-text
cleaner = re.compile('<.*?>')
cleaner3 = re.compile('\,\,\,')
cleaner4 = re.compile("[\.][\,][\,]")
cleaner5 = re.compile("[\,][\,]")
cleaner6 = re.compile("[\;][\,]")

In [ ]:
mypath = './uspto'
paths = [mypath+'/'+f for f in listdir(mypath) if isfile(join(mypath, f))]
paths = [paths[0]]
df = pd.DataFrame(columns=['grant_id','description_text'])
for path in paths:
    print(path)
    file = open(file=path, mode='r') #opening the file in read mode
    file_content_raw = file.read()
    file.close()
    text1 = re.compile("<\?xml version=\"1\.0\" encoding\=\"UTF\-8\"\?>")
    file_content = text1.split(file_content_raw)
    while '' in file_content:
        file_content.remove('')
    gid_list = []
    description_list = []
    for line in file_content:
        
        gid = grant_id.findall(line)
        # For description_text
        description_text = ''
        try:
            tree = ET.ElementTree(ET.fromstring(line))
            description = tree.find('description')
        except:
            description = None
        if description is None:
            pass
        else:
            for child in description.iter():
                if child.text == None:
                    pass

                elif child.tag == 'p' or child.tag == 'description of drawings':
                    description_text = description_text + " \n ========== \n " + child.text  + ' \n '
                elif child.tag == 'heading':
                    description_text =  description_text + " \n ========== \n " + child.text + ' \n '

                else:
                    description_text = description_text + child.text
            # cleaning description text
            description_text = re.sub(cleaner,'',description_text)
            description_text = re.sub(cleaner3,'',description_text)
            description_text = re.sub(cleaner4,'',description_text)
            description_text = re.sub(cleaner5,'',description_text)
            description_text = re.sub(cleaner6,'',description_text)
            description_text = " ========== " + description_text + " ========== "
        if len(gid)!=0: 
            gid_list.append(gid[0])
            description_list.append(description_text)
    
    # Get all results ad DataFrame
    data_frame = pd.DataFrame(
        {'grant_id': gid_list,
         'description_text': description_list
        })
    #print(data_frame)
    df = df.append(data_frame, ignore_index=True)

In [71]:
df

,grant_id,description_text
0,USD0961886,========== \n\n\n \n ========== \n 1. Candy \...
1,USD0961887,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....
2,USD0961888,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....
3,USD0961889,========== \n\n\nFIG. 1FIG. 2FIG. 1FIG. 3FIG....
4,USD0961890,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....
...,...,...
7257,US11432447,========== \n\n \n ========== \n BACKGROUND O...
7258,US11432448,========== \n\n \n ========== \n CROSS-REFERE...
7259,US11432449,========== \n\n \n ========== \n CROSS-REFERE...
7260,US11432450,========== \n\n \n ========== \n TECHNICAL FI...


In [73]:
df.to_csv('./uspto_1_desc.csv')

# Preparing the dataset for supervised segmentation

In [5]:
df = pd.read_csv('./uspto_1_desc.csv')

In [6]:
def get_text_segment(row):
    text = row['description_text'].split('\n')
    sentences = []
    sentences_binary = []
    for i in range(1,len(text)):
        if text[i] == ' ========== ' :
            sentences_binary[-1] = 1
        else:
            sentences.append(text[i])
            sentences_binary.append(0)

    #clean irrelevant text
    index_remove = [idx for idx, x in enumerate(sentences) if x.strip() == "" ]
    if index_remove:
        sentences = [ele for idx, ele in enumerate(sentences) if idx not in index_remove]
        sentences_binary = [ele for idx, ele in enumerate(sentences_binary) if idx not in index_remove]
    return sentences, sentences_binary
    
    


In [ ]:
output = df.apply(get_text_segment,axis =1)
df[['segments',' segments_binary']] = pd.DataFrame(output.to_list(), index=df.index)

In [10]:
df

,Unnamed: 0,grant_id,description_text,segments,segments_binary
0,0,USD0961886,========== \n\n\n \n ========== \n 1. Candy \...,"[ 1. Candy , 1.11.21.31.41.51.61.7 , The das...","[0, 1, 0, 0]"
1,1,USD0961887,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....,[FIG. 1FIG. 2FIG. 3FIG. 4FIG. 5FIG. 6FIG. 7FIG...,"[1, 0, 0]"
2,2,USD0961888,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....,[FIG. 1FIG. 2FIG. 3FIG. 4FIG. 5FIG. 6FIG. 7FIG...,"[1, 0, 0]"
3,3,USD0961889,========== \n\n\nFIG. 1FIG. 2FIG. 1FIG. 3FIG....,[FIG. 1FIG. 2FIG. 1FIG. 3FIG. 1FIG. 4FIG. 1FIG...,"[1, 0, 1, 0, 1, 0, 1, 0, 0]"
4,4,USD0961890,========== \n\n\nFIG. 1FIG. 2FIG. 3FIG. 4FIG....,[FIG. 1FIG. 2FIG. 3FIG. 4FIG. 5FIG. 6FIG. 7FIG...,[0]
...,...,...,...,...,...
7257,7257,US11432447,========== \n\n \n ========== \n BACKGROUND O...,"[ BACKGROUND OF THE INVENTION , Field of the ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7258,7258,US11432448,========== \n\n \n ========== \n CROSS-REFERE...,"[ CROSS-REFERENCE TO RELATED APPLICATIONS , T...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7259,7259,US11432449,========== \n\n \n ========== \n CROSS-REFERE...,"[ CROSS-REFERENCE TO RELATED APPLICATIONS , T...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7260,7260,US11432450,========== \n\n \n ========== \n TECHNICAL FI...,"[ TECHNICAL FIELD , The present disclosure re...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
df.to_csv('./uspto_1_desc_final.csv')